## Construction d'un pipeline MLlib

In [1]:
from pyspark.sql import functions as F
from modules.spark import spark

In [9]:
df = spark.read.csv("../data/processed/data-balanced", header=True, inferSchema=True)


### Assembler le pipeline MLlib.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import pipeline

indexers = [
    StringIndexer(inputCol="Type", outputCol="Type_index"),
    StringIndexer(inputCol="ShippingMode", outputCol="ShippingMode_index"),
    StringIndexer(inputCol="CategoryName", outputCol="CategoryName_index"),
    StringIndexer(inputCol="CustomerSegment", outputCol="CustomerSegment_index"),
    StringIndexer(inputCol="OrderRegion", outputCol="OrderRegion_index"),
    StringIndexer(inputCol="ShippingMonthName", outputCol="ShippingMonthName_index"),
]
encoder = OneHotEncoder(
    inputCols=['Type_index', 'ShippingMode_index', 'CategoryName_index', 'CustomerSegment_index', 'OrderRegion_index', 'ShippingMonthName_index'],
    outputCols=['Type_vec', 'ShippingMode_vec', 'CategoryName_vec', 'CustomerSegment_vec', 'OrderRegion_vec', 'ShippingMonthName_vec']
)

assembler = VectorAssembler(
    inputCols=['OrderItemTotal', 'Type_vec', 'ShippingMode_vec', 'CategoryName_vec', 'CustomerSegment_vec', 'OrderRegion_vec', 'ShippingMonthName_vec'],
    outputCol='features'
)

pl = pipeline(stage=indexers + [encoder, assembler])



